In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import kmapper as km
import pyballmapper as pbm
import statistics
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib import colormaps as cm
import networkx as nx

/Users/elliebarranbeck/ERP clean/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Election Data

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ

In [2]:
# County Presidential Election Returns 2000-2020
df=pd.read_csv('US_data/countypres_2000-2020.csv')

In [3]:
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [48]:
## Extracting the useful columns 
cols = ['year',
        'state',
        'county_name',
        'county_fips',
         'party',
         'candidatevotes'
         ,'totalvotes']

data = df[cols]
#calculating vote percentages
data['percentagevotes']=data['candidatevotes']/data['totalvotes']*100
pv = data.pivot_table(index=['county_fips','county_name','state'], columns=['party','year'], values='percentagevotes')
pv=pv.reset_index()

/var/folders/xf/8y4gbp61673b4f2br89yscz40000gn/T/ipykernel_37195/2920509478.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['percentagevotes']=data['candidatevotes']/data['totalvotes']*100


In [5]:
pv.columns = [f'{party}_{year}' if year else f'{party}' for party, year in pv.columns]
# Reset index
pv = pv.reset_index()
pv

,index,county_fips,county_name,state,DEMOCRAT_2000,DEMOCRAT_2004,DEMOCRAT_2008,DEMOCRAT_2012,DEMOCRAT_2016,DEMOCRAT_2020,...,OTHER_2008,OTHER_2012,OTHER_2016,OTHER_2020,REPUBLICAN_2000,REPUBLICAN_2004,REPUBLICAN_2008,REPUBLICAN_2012,REPUBLICAN_2016,REPUBLICAN_2020
0,0,1001.0,AUTAUGA,ALABAMA,28.719200,23.694039,25.773021,26.587832,23.769671,27.018365,...,0.613341,0.793916,3.463741,1.544833,69.694328,75.673522,73.613637,72.618252,72.766588,71.436802
1,1,1003.0,BALDWIN,ALABAMA,24.782224,22.502885,23.811922,21.589444,19.385601,22.409030,...,0.928599,1.052286,4.068687,1.419597,72.365439,76.415176,75.259479,77.358269,76.545712,76.171373
2,2,1005.0,BARBOUR,ALABAMA,49.908610,44.836225,48.985383,51.368494,46.527844,45.788173,...,0.576096,0.408376,1.375490,0.760601,49.023569,54.736940,50.438521,48.223130,52.096666,53.451226
3,3,1007.0,BIBB,ALABAMA,38.163639,27.486842,26.596483,26.152019,21.249575,20.698280,...,0.960204,1.021378,2.347205,0.875456,60.174623,72.000000,72.443313,72.826603,76.403220,78.426264
4,4,1009.0,BLOUNT,ALABAMA,27.691537,18.312872,14.513537,12.371907,8.425825,9.569378,...,1.467013,1.162209,2.239331,0.859069,70.477939,80.850074,84.019450,86.465884,89.334844,89.571553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,3190,56039.0,TETON,WYOMING,38.529384,52.575051,60.457966,54.195743,57.923497,66.599040,...,2.605389,3.428123,11.023996,2.319605,52.286454,45.109605,36.936645,42.376134,31.052507,29.356868
3191,3191,56041.0,UINTA,WYOMING,22.243192,22.460092,27.524353,19.065464,14.191263,16.819960,...,4.015206,3.466448,13.152302,2.114388,73.726072,75.250588,68.460442,77.468088,72.656434,79.247278
3192,3192,56043.0,WASHAKIE,WYOMING,19.896322,20.782693,25.414634,20.131846,13.948610,16.145833,...,2.487805,3.448276,9.727320,1.760913,77.462355,77.783179,72.097561,76.419878,76.324069,80.481151
3193,3193,56045.0,WESTON,WYOMING,14.673203,17.099057,19.273579,12.563263,8.479864,10.112360,...,4.042179,3.453409,5.501985,1.320225,82.385621,80.748821,76.684241,83.983328,86.018151,87.275281


In [49]:
pv['GEO_ID'] = pv['county_fips'].astype(int)

In [50]:
elec_data=pv[['GEO_ID','county_name','state','DEMOCRAT_2012','REPUBLICAN_2012','DEMOCRAT_2016','REPUBLICAN_2016']]

KeyError: "['DEMOCRAT_2012' 'REPUBLICAN_2012' 'DEMOCRAT_2016' 'REPUBLICAN_2016'] not in index"

# Demographic Data

Census and American Community Survey from 2016 using 5 year estimates
https://data.census.gov/

In [8]:
elec_data.to_csv('US results/Output datasets/election_data_US.csv')

## Education

In [10]:
edu_data=pd.read_csv('US_data/education.csv')

In [11]:
edu_data

,GEO_ID,Nohighschool_1824,Highschool_1824,Somedegree_1824,Degree_1824,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus
0,0500000US01001,18.8,37.5,36.2,7.4,12.4,34.3,21.5,7.1,24.6
1,0500000US01003,18.7,31.9,41.3,8.1,9.9,28.7,22.4,9.4,29.5
2,0500000US01005,21.6,44.2,30.7,3.5,26.3,34.9,17.8,8.2,12.9
3,0500000US01007,25.5,31.1,42.0,1.5,19.3,41.8,21.1,5.8,12.0
4,0500000US01009,18.2,32.8,46.1,2.9,19.9,32.9,22.5,11.6,13.0
...,...,...,...,...,...,...,...,...,...,...
3215,0500000US72145,26.2,10.7,59.8,3.3,41.8,13.7,12.5,11.3,20.7
3216,0500000US72147,6.1,28.8,44.8,20.2,30.7,35.8,7.0,7.5,19.0
3217,0500000US72149,8.5,32.5,53.2,5.8,29.6,34.5,10.2,7.2,18.6
3218,0500000US72151,12.8,27.8,53.5,5.9,32.5,26.1,13.3,11.2,16.8


In [12]:
edu_data=edu_data[['GEO_ID','Nohighschool_25plus', 'Highschool_25plus',
       'Somedegree_25plus', 'Assocdegree_25plus', 'Degree_25plus']]

## Age

In [13]:
age_data=pd.read_csv('US_data/age.csv')

In [14]:
age_data.describe()

,Under18,18to24,25to29,30to34,35to39,40to44,45to49,50to54,55to59,60to64,Over65
count,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000
mean,22.574099,9.037640,5.797329,5.877950,5.711553,5.976988,6.344441,7.201988,7.273199,6.654969,17.548230
std,3.459112,3.526647,1.342807,1.107852,1.018101,1.012998,0.880416,0.922570,1.294687,1.341200,4.459395
min,1.100000,0.700000,0.000000,0.000000,0.000000,1.100000,0.300000,0.000000,1.800000,1.400000,3.900000
25%,20.500000,7.400000,5.000000,5.300000,5.100000,5.400000,5.900000,6.800000,6.500000,5.800000,14.700000
50%,22.500000,8.300000,5.700000,5.800000,5.700000,6.000000,6.400000,7.200000,7.200000,6.500000,17.200000
75%,24.300000,9.600000,6.400000,6.400000,6.300000,6.600000,6.900000,7.700000,7.900000,7.300000,19.825000
max,40.300000,55.700000,15.200000,15.100000,10.200000,15.800000,12.500000,13.900000,31.600000,15.100000,53.100000


In [15]:
age_data['25to34']=age_data['25to29']+age_data['30to34']
age_data['35to49']=age_data['35to39']+age_data['40to44']+age_data['45to49']
age_data['50to64']=age_data['50to54']+age_data['55to59']+age_data['60to64']

In [16]:
age_data.columns

Index(['GEO_ID', 'Under18', '18to24', '25to29', '30to34', '35to39', '40to44',
       '45to49', '50to54', '55to59', '60to64', 'Over65', '25to34', '35to49',
       '50to64'],
      dtype='object')

In [17]:
age_data=age_data[['GEO_ID', 'Under18', '18to24', '25to34', '35to49',
       '50to64','Over65', ]]

### Ethnicity

In [18]:
eth_data=pd.read_csv('US_data/ethnicity.csv')

In [19]:
eth_data

,GEO_ID,White,Black
0,0500000US01001,76.7,20.9
1,0500000US01003,88.0,9.0
2,0500000US01005,47.3,48.6
3,0500000US01007,77.5,20.8
4,0500000US01009,91.7,2.1
...,...,...,...
3216,0500000US72145,59.4,16.6
3217,0500000US72147,35.8,26.0
3218,0500000US72149,58.3,16.0
3219,0500000US72151,43.9,20.1


### Rural/Urban

In [20]:
rural_data=pd.read_csv('US_data/rural_urban.csv')

In [21]:
rural_data['Rural']=(rural_data['Rural']/rural_data['Tot_pop '])*100

In [22]:
rural_data=rural_data[['GEO_ID','Rural']]

In [23]:
rural_data.describe()

,Rural
count,3221.000000
mean,62.770482
std,34.341945
min,0.000000
25%,33.509863
50%,65.105101
75%,100.000000
max,100.000000


### Income data

In [24]:
income_data=pd.read_csv('US_Data/income_cat.csv')
income_data['GEO_ID']=income_data['GEO_ID'].astype(object)


In [25]:
income_data

,GEO_ID,lessthan50K,50to75k,75to100k,morethan100k
0,1001,46.4,19.5,13.2,20.8
1,1003,48.5,18.6,12.0,20.9
2,1005,65.9,14.6,9.4,9.9
3,1007,60.3,17.7,11.4,10.6
4,1009,53.9,21.8,11.0,13.3
...,...,...,...,...,...
3109,56037,36.3,18.6,15.1,30.0
3110,56039,29.4,20.3,14.3,36.0
3111,56041,47.7,16.4,13.3,22.5
3112,56043,53.4,20.4,11.9,14.3


In [26]:
income_data.describe()

,lessthan50K,50to75k,75to100k,morethan100k
count,3114.000000,3114.000000,3114.000000,3114.000000
mean,52.949615,18.539788,11.672319,16.839274
std,10.404013,2.787136,2.709991,7.932256
min,14.800000,6.600000,1.300000,2.500000
25%,46.500000,16.800000,9.900000,11.600000
50%,53.400000,18.600000,11.900000,15.000000
75%,60.300000,20.300000,13.500000,19.800000
max,83.000000,30.200000,32.400000,63.000000


### Industry data

In [27]:
industry_data=pd.read_csv('US_data/industry.csv')

In [28]:
industry_data.describe()

,manu_arg
count,3220.000000
mean,19.032764
std,8.307970
min,0.200000
25%,12.800000
50%,18.500000
75%,24.500000
max,69.600000


### Unemployment data

In [29]:
unemp_data=pd.read_csv('US_data/unemployment.csv')

In [30]:
poverty_data=pd.read_csv('US_data/poverty.csv')

In [31]:
poverty_data

,GEO_ID,Poverty
0,0500000US01001,12.3
1,0500000US01003,13.0
2,0500000US01005,26.4
3,0500000US01007,16.5
4,0500000US01009,16.5
...,...,...
3215,0500000US72145,46.0
3216,0500000US72147,35.9
3217,0500000US72149,50.8
3218,0500000US72151,49.5


In [32]:
health_data=pd.read_csv('US_data/countyhealth.csv')
health_data=health_data.dropna()
health_data['FIPS']=health_data['FIPS'].astype(object)
health_data[['Adult obesity raw value','Poor or fair health raw value','Unemployment raw value']]=health_data[['Adult obesity raw value','Poor or fair health raw value','Unemployment raw value']]*100

In [33]:
health_data.tail()

,FIPS,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value
3187,56037,29.2,13.9,4.162206
3188,56039,12.5,11.1,4.795996
3189,56041,29.6,14.7,5.034880
3190,56043,24.7,14.3,4.511628
3191,56045,29.4,13.4,3.425693


In [34]:
health_data.describe()

,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value
count,3190.000000,3190.000000,3190.000000
mean,30.903301,16.913346,6.258686
std,4.467052,4.951684,2.287594
min,10.700000,7.200000,1.188560
25%,28.500000,13.000000,4.674522
50%,31.100000,15.900000,6.033636
75%,33.675000,20.000000,7.500364
max,46.600000,41.700000,23.668430


## Merged dataset

In [35]:
demo_data = (edu_data
             .merge(age_data, on='GEO_ID')
             .merge(eth_data, on='GEO_ID')
             .merge(rural_data, on='GEO_ID')
             .merge(industry_data, on='GEO_ID')
             .merge(poverty_data,on='GEO_ID'))


In [36]:
demo_data

,GEO_ID,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty
0,0500000US01001,12.4,34.3,21.5,7.1,24.6,25.2,8.6,12.2,21.0,19.1,14.0,76.7,20.9,41.030801,15.2,12.3
1,0500000US01003,9.9,28.7,22.4,9.4,29.5,22.2,7.7,11.3,19.1,21.0,18.7,88.0,9.0,32.815672,10.6,13.0
2,0500000US01005,26.3,34.9,17.8,8.2,12.9,21.5,9.4,13.8,19.1,19.7,16.5,47.3,48.6,64.882080,26.9,26.4
3,0500000US01007,19.3,41.8,21.1,5.8,12.0,21.1,9.2,13.8,21.4,19.7,14.9,77.5,20.8,100.000000,25.0,16.5
4,0500000US01009,19.9,32.9,22.5,11.6,13.0,23.6,8.0,11.5,19.8,19.9,17.2,91.7,2.1,90.719682,18.5,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214,0500000US72145,41.8,13.7,12.5,11.3,20.7,22.2,9.8,12.7,19.3,19.0,17.1,59.4,16.6,5.602611,17.2,46.0
3215,0500000US72147,30.7,35.8,7.0,7.5,19.0,21.7,9.7,9.8,18.0,21.2,19.6,35.8,26.0,21.157835,7.3,35.9
3216,0500000US72149,29.6,34.5,10.2,7.2,18.6,24.2,11.4,12.2,18.4,19.7,14.1,58.3,16.0,17.892958,21.5,50.8
3217,0500000US72151,32.5,26.1,13.3,11.2,16.8,22.0,9.9,11.7,19.1,20.7,16.5,43.9,20.1,16.971225,18.9,49.5


In [37]:
demo_data['GEO_ID'] = demo_data['GEO_ID'].str.replace('0500000US', '', regex=False).astype(int)
demo_data=demo_data.merge(income_data, on='GEO_ID')

In [38]:
demo_data

,GEO_ID,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,...,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k
0,1001,12.4,34.3,21.5,7.1,24.6,25.2,8.6,12.2,21.0,...,14.0,76.7,20.9,41.030801,15.2,12.3,46.4,19.5,13.2,20.8
1,1003,9.9,28.7,22.4,9.4,29.5,22.2,7.7,11.3,19.1,...,18.7,88.0,9.0,32.815672,10.6,13.0,48.5,18.6,12.0,20.9
2,1005,26.3,34.9,17.8,8.2,12.9,21.5,9.4,13.8,19.1,...,16.5,47.3,48.6,64.882080,26.9,26.4,65.9,14.6,9.4,9.9
3,1007,19.3,41.8,21.1,5.8,12.0,21.1,9.2,13.8,21.4,...,14.9,77.5,20.8,100.000000,25.0,16.5,60.3,17.7,11.4,10.6
4,1009,19.9,32.9,22.5,11.6,13.0,23.6,8.0,11.5,19.8,...,17.2,91.7,2.1,90.719682,18.5,16.5,53.9,21.8,11.0,13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,56037,9.3,34.1,24.9,10.3,21.4,27.5,9.1,15.5,18.7,...,9.4,91.1,1.6,12.071037,25.1,11.0,36.3,18.6,15.1,30.0
3110,56039,4.6,15.8,19.5,6.2,53.8,19.5,5.7,20.0,22.7,...,11.8,91.4,0.5,62.744654,6.9,7.3,29.4,20.3,14.3,36.0
3111,56041,10.4,36.6,26.1,8.8,18.2,29.6,8.0,12.7,18.6,...,10.7,93.6,0.8,42.521028,15.5,16.8,47.7,16.4,13.3,22.5
3112,56043,12.5,30.8,27.0,8.6,21.1,24.8,5.3,11.3,17.3,...,19.7,93.1,0.7,34.398545,22.8,14.7,53.4,20.4,11.9,14.3


In [39]:
demo_data=demo_data.merge(health_data, left_on='GEO_ID',right_on='FIPS')

In [40]:
demo_data.describe()

,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,...,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value
count,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,...,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000,3114.000000
mean,14.224277,34.587091,21.880861,8.525498,20.783847,22.558703,9.009955,11.634586,18.002216,21.168433,...,63.754916,19.218915,16.418979,52.949615,18.539788,11.672319,16.839274,30.947559,16.923892,6.228573
std,6.542912,7.070799,3.792383,2.555754,9.139611,3.422010,3.573370,2.247991,2.122785,2.797690,...,33.772956,8.295985,6.505763,10.404013,2.787136,2.709991,7.932256,4.468338,4.959968,2.233174
min,1.300000,6.500000,8.300000,0.600000,3.000000,3.900000,0.700000,0.000000,4.900000,5.200000,...,0.000000,0.200000,1.800000,14.800000,6.600000,1.300000,2.500000,10.700000,7.200000,1.188560
25%,9.300000,30.200000,19.325000,6.800000,14.400000,20.500000,7.300000,10.300000,16.700000,19.600000,...,35.146463,13.000000,11.800000,46.500000,16.800000,9.900000,11.600000,28.500000,13.000000,4.660904
50%,12.800000,34.900000,21.800000,8.300000,18.500000,22.500000,8.200000,11.400000,18.100000,21.200000,...,66.050346,18.700000,15.700000,53.400000,18.600000,11.900000,15.000000,31.200000,15.900000,6.018293
75%,18.300000,39.600000,24.300000,10.100000,24.700000,24.300000,9.500000,12.700000,19.200000,22.700000,...,100.000000,24.700000,19.900000,60.300000,20.300000,13.500000,19.800000,33.700000,20.000000,7.489068
max,51.500000,54.600000,36.300000,19.500000,80.200000,40.300000,55.700000,26.900000,29.400000,44.800000,...,100.000000,69.600000,48.700000,83.000000,30.200000,32.400000,63.000000,46.600000,41.700000,23.617721


In [41]:
demo_data.columns

Index(['GEO_ID', 'Nohighschool_25plus', 'Highschool_25plus',
       'Somedegree_25plus', 'Assocdegree_25plus', 'Degree_25plus', 'Under18',
       '18to24', '25to34', '35to49', '50to64', 'Over65', 'White', 'Black',
       'Rural', 'manu_arg', 'Poverty', 'lessthan50K', '50to75k', '75to100k',
       'morethan100k', 'FIPS', 'Adult obesity raw value',
       'Poor or fair health raw value', 'Unemployment raw value'],
      dtype='object')

In [42]:
#Output to latex
summary=demo_data.drop(['FIPS','GEO_ID'],axis=1 ).describe()
summary=summary.transpose()
summary=summary.round(2)
summary = summary.applymap(lambda x: '{:.2f}'.format(x).rstrip('0').rstrip('.'))
latex_code = summary.to_latex(index=True)
print(latex_code)

\begin{tabular}{lllllllll}
\toprule
 & count & mean & std & min & 25% & 50% & 75% & max \\
\midrule
Nohighschool_25plus & 3114 & 14.22 & 6.54 & 1.3 & 9.3 & 12.8 & 18.3 & 51.5 \\
Highschool_25plus & 3114 & 34.59 & 7.07 & 6.5 & 30.2 & 34.9 & 39.6 & 54.6 \\
Somedegree_25plus & 3114 & 21.88 & 3.79 & 8.3 & 19.32 & 21.8 & 24.3 & 36.3 \\
Assocdegree_25plus & 3114 & 8.53 & 2.56 & 0.6 & 6.8 & 8.3 & 10.1 & 19.5 \\
Degree_25plus & 3114 & 20.78 & 9.14 & 3 & 14.4 & 18.5 & 24.7 & 80.2 \\
Under18 & 3114 & 22.56 & 3.42 & 3.9 & 20.5 & 22.5 & 24.3 & 40.3 \\
18to24 & 3114 & 9.01 & 3.57 & 0.7 & 7.3 & 8.2 & 9.5 & 55.7 \\
25to34 & 3114 & 11.63 & 2.25 & 0 & 10.3 & 11.4 & 12.7 & 26.9 \\
35to49 & 3114 & 18 & 2.12 & 4.9 & 16.7 & 18.1 & 19.2 & 29.4 \\
50to64 & 3114 & 21.17 & 2.8 & 5.2 & 19.6 & 21.2 & 22.7 & 44.8 \\
Over65 & 3114 & 17.62 & 4.45 & 3.9 & 14.8 & 17.3 & 19.9 & 53.1 \\
White & 3114 & 83.53 & 16.41 & 10.1 & 75.8 & 89.7 & 96.1 & 99.5 \\
Black & 3114 & 9.97 & 14.48 & 0.2 & 1.2 & 3.3 & 12 & 88.6 \\
Rural 

/var/folders/xf/8y4gbp61673b4f2br89yscz40000gn/T/ipykernel_37195/3538990198.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary = summary.applymap(lambda x: '{:.2f}'.format(x).rstrip('0').rstrip('.'))


In [43]:
US_full_dataset=demo_data.merge(elec_data, on='GEO_ID')

In [44]:
US_full_dataset

,GEO_ID,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,...,FIPS,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,county_name,state,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016
0,1001,12.4,34.3,21.5,7.1,24.6,25.2,8.6,12.2,21.0,...,1001,30.9,19.4,5.883047,AUTAUGA,ALABAMA,26.587832,72.618252,23.769671,72.766588
1,1003,9.9,28.7,22.4,9.4,29.5,22.2,7.7,11.3,19.1,...,1003,26.7,16.0,6.053720,BALDWIN,ALABAMA,21.589444,77.358269,19.385601,76.545712
2,1005,26.3,34.9,17.8,8.2,12.9,21.5,9.4,13.8,19.1,...,1005,40.8,25.7,10.809275,BARBOUR,ALABAMA,51.368494,48.223130,46.527844,52.096666
3,1007,19.3,41.8,21.1,5.8,12.0,21.1,9.2,13.8,21.4,...,1007,40.1,22.0,7.137850,BIBB,ALABAMA,26.152019,72.826603,21.249575,76.403220
4,1009,19.9,32.9,22.5,11.6,13.0,23.6,8.0,11.5,19.8,...,1009,32.4,20.7,6.146908,BLOUNT,ALABAMA,12.371907,86.465884,8.425825,89.334844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,56037,9.3,34.1,24.9,10.3,21.4,27.5,9.1,15.5,18.7,...,56037,29.2,13.9,4.162206,SWEETWATER,WYOMING,28.256881,67.641314,18.861646,70.951547
3110,56039,4.6,15.8,19.5,6.2,53.8,19.5,5.7,20.0,22.7,...,56039,12.5,11.1,4.795996,TETON,WYOMING,54.195743,42.376134,57.923497,31.052507
3111,56041,10.4,36.6,26.1,8.8,18.2,29.6,8.0,12.7,18.6,...,56041,29.6,14.7,5.034880,UINTA,WYOMING,19.065464,77.468088,14.191263,72.656434
3112,56043,12.5,30.8,27.0,8.6,21.1,24.8,5.3,11.3,17.3,...,56043,24.7,14.3,4.511628,WASHAKIE,WYOMING,20.131846,76.419878,13.948610,76.324069


In [45]:
summary=US_full_dataset.describe()
summary=summary.transpose()
summary=summary.round(2)
summary = summary.applymap(lambda x: '{:.2f}'.format(x).rstrip('0').rstrip('.'))
latex_code = summary.to_latex(index=True)
print(latex_code)

\begin{tabular}{lllllllll}
\toprule
 & count & mean & std & min & 25% & 50% & 75% & max \\
\midrule
Nohighschool_25plus & 3114 & 14.22 & 6.54 & 1.3 & 9.3 & 12.8 & 18.3 & 51.5 \\
Highschool_25plus & 3114 & 34.59 & 7.07 & 6.5 & 30.2 & 34.9 & 39.6 & 54.6 \\
Somedegree_25plus & 3114 & 21.88 & 3.79 & 8.3 & 19.32 & 21.8 & 24.3 & 36.3 \\
Assocdegree_25plus & 3114 & 8.53 & 2.56 & 0.6 & 6.8 & 8.3 & 10.1 & 19.5 \\
Degree_25plus & 3114 & 20.78 & 9.14 & 3 & 14.4 & 18.5 & 24.7 & 80.2 \\
Under18 & 3114 & 22.56 & 3.42 & 3.9 & 20.5 & 22.5 & 24.3 & 40.3 \\
18to24 & 3114 & 9.01 & 3.57 & 0.7 & 7.3 & 8.2 & 9.5 & 55.7 \\
25to34 & 3114 & 11.63 & 2.25 & 0 & 10.3 & 11.4 & 12.7 & 26.9 \\
35to49 & 3114 & 18 & 2.12 & 4.9 & 16.7 & 18.1 & 19.2 & 29.4 \\
50to64 & 3114 & 21.17 & 2.8 & 5.2 & 19.6 & 21.2 & 22.7 & 44.8 \\
Over65 & 3114 & 17.62 & 4.45 & 3.9 & 14.8 & 17.3 & 19.9 & 53.1 \\
White & 3114 & 83.53 & 16.41 & 10.1 & 75.8 & 89.7 & 96.1 & 99.5 \\
Black & 3114 & 9.97 & 14.48 & 0.2 & 1.2 & 3.3 & 12 & 88.6 \\
Rural 

/var/folders/xf/8y4gbp61673b4f2br89yscz40000gn/T/ipykernel_37195/769151765.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary = summary.applymap(lambda x: '{:.2f}'.format(x).rstrip('0').rstrip('.'))


In [46]:
US_full_dataset.to_csv('US results/Outputs/US_fulldataset.csv',index=False)

OSError: Cannot save file into a non-existent directory: 'US results/Outputs'

In [ ]:
columns=['REPUBLICAN_2016']
ball_rep=pd.DataFrame()
for col in columns:
    ball_rep[col]=US_results.groupby('ball')[col].mean()
ball_rep['Size'] = US_results.groupby('ball').size()

NameError: name 'US_results' is not defined

In [ ]:
ball_rep

,REPUBLICAN_2016,Size
ball,,
0,63.292547,926
1,56.907903,156
2,72.156954,922
3,37.005677,47
4,55.738564,174
5,69.101354,901
6,46.448863,251
7,36.035029,24
8,61.656207,1


In [ ]:
ball_rep[ball_rep['REPUBLICAN_2016']<63].describe()

,REPUBLICAN_2016,Size
count,33.000000,33.000000
mean,47.184428,91.212121
std,11.674933,93.550707
min,25.860396,1.000000
25%,37.213778,18.000000
50%,49.231496,53.000000
75%,57.549769,139.000000
max,61.656207,355.000000


In [ ]:
ball_rep[ball_rep['REPUBLICAN_2016']>63].describe()

,REPUBLICAN_2016,Size
count,26.000000,26.000000
mean,71.648567,297.576923
std,5.889065,312.913684
min,63.292547,1.000000
25%,66.288186,38.250000
50%,72.484009,169.500000
75%,75.301374,533.750000
max,89.230769,926.000000


## Analysing the balls

In [ ]:
pd.set_option('display.max_columns', 1000)
US_results[(US_results['ball']==45)&(US_results['REPUBLICAN_2012']<US_results['DEMOCRAT_2012'])].describe()

,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016,vote_gains,seat_gains,point,ball
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.0
mean,9.925000,38.022917,21.585417,11.060417,19.416667,21.591667,8.939583,10.993750,17.814583,22.220833,18.452083,94.672917,2.464583,60.181444,22.906250,13.014583,49.697917,20.441667,13.260417,16.597917,30.933333,13.025000,5.785313,54.090252,43.194998,40.056687,52.938076,9.743078,68.750000,1612.812500,45.0
std,2.452224,3.636589,2.689538,1.780717,3.856431,2.194076,2.953738,0.987455,1.311324,1.585165,2.273528,3.163202,1.452875,12.543692,5.093943,3.054191,5.023752,1.359260,1.736681,3.371374,2.885572,1.746303,1.277994,3.401839,3.618061,4.325821,4.730064,3.642752,46.841744,896.551304,0.0
min,4.700000,33.200000,15.000000,7.100000,13.100000,15.600000,6.500000,9.200000,15.200000,18.500000,13.900000,85.600000,0.800000,36.583679,11.900000,7.300000,37.400000,16.900000,8.100000,9.700000,23.800000,10.100000,3.826943,47.599047,32.267745,31.790421,39.521320,1.963527,0.000000,596.000000,45.0
25%,7.950000,34.975000,19.900000,10.000000,16.475000,20.200000,7.200000,10.375000,16.800000,21.575000,16.875000,93.325000,1.300000,49.242589,19.750000,11.025000,46.000000,19.700000,12.400000,14.350000,29.675000,11.800000,4.786968,50.971886,41.033920,37.185297,49.867708,7.439353,0.000000,813.250000,45.0
50%,10.000000,37.850000,21.350000,11.200000,19.000000,21.950000,7.900000,10.900000,18.000000,21.900000,18.500000,95.750000,2.100000,63.334728,22.800000,13.050000,50.350000,20.550000,13.150000,16.650000,30.700000,12.700000,5.432681,53.968567,42.567064,39.265355,53.761373,10.212712,100.000000,1304.000000,45.0
75%,11.825000,39.850000,22.900000,12.125000,21.675000,22.925000,8.700000,11.725000,18.825000,22.825000,19.825000,97.025000,3.350000,70.533247,25.450000,15.325000,52.775000,21.225000,14.150000,18.625000,32.600000,14.125000,6.695518,56.731441,46.442118,43.077868,56.459130,12.670349,100.000000,2117.500000,45.0
max,14.600000,51.100000,28.200000,14.500000,30.200000,26.700000,19.100000,13.300000,20.600000,25.700000,23.300000,98.700000,5.700000,84.891045,37.900000,20.600000,64.400000,24.100000,17.600000,26.300000,38.500000,17.500000,8.740741,60.624497,49.154917,53.756426,60.911540,17.975555,100.000000,3081.000000,45.0


In [ ]:
US_results[(US_results['ball']==45)&(US_results['REPUBLICAN_2016']<US_results['DEMOCRAT_2016'])].describe()

,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016,vote_gains,seat_gains,point,ball
count,4.000000,4.000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0,4.000000,4.0
mean,9.675000,36.525,19.575000,9.600000,24.650000,20.750000,8.850000,11.375000,19.275000,22.775000,16.975000,96.050000,2.200000,68.178747,17.900000,11.375000,45.600000,20.075000,14.175000,20.200000,28.125000,11.500000,5.134531,59.373390,36.885456,48.096041,42.797934,5.912478,0.0,2197.500000,45.0
std,2.970269,1.550,1.510794,0.816497,3.738538,2.716002,3.468429,1.014479,1.037224,1.291962,3.617895,2.000833,1.867262,6.210086,4.726521,3.484609,3.652396,0.556028,1.811767,2.264214,3.086935,1.321615,1.125503,1.146267,3.462159,4.203800,3.068529,4.710096,0.0,873.533247,0.0
min,7.000000,34.900,18.400000,8.400000,22.200000,18.300000,6.500000,10.400000,18.100000,21.300000,13.900000,93.100000,1.200000,63.427576,14.300000,8.600000,42.000000,19.500000,12.400000,18.400000,23.800000,10.600000,4.104232,57.996067,32.267745,43.673813,39.521320,2.144241,0.0,1156.000000,45.0
25%,7.600000,35.350,18.550000,9.450000,22.575000,18.450000,7.025000,10.550000,18.775000,22.125000,14.800000,95.725000,1.275000,63.884105,14.450000,8.825000,42.825000,19.650000,13.075000,18.400000,27.100000,10.600000,4.450651,58.712221,35.992067,46.097553,40.537469,3.268068,0.0,1646.500000,45.0
50%,9.050000,36.650,19.100000,9.900000,23.100000,20.750000,7.450000,11.400000,19.200000,22.700000,15.950000,96.800000,1.300000,66.197456,16.450000,10.400000,45.350000,20.100000,13.850000,19.650000,28.900000,11.000000,4.872886,59.436498,37.305293,47.476963,42.940818,4.383983,0.0,2296.500000,45.0
75%,11.125000,37.825,20.125000,10.050000,25.175000,23.050000,9.275000,12.225000,19.700000,23.350000,18.125000,97.125000,2.225000,70.492098,19.900000,12.950000,48.125000,20.525000,14.950000,21.450000,29.925000,11.900000,5.556766,60.097666,38.198682,49.475451,45.201283,7.028393,0.0,2847.500000,45.0
max,13.600000,37.900,21.700000,10.200000,30.200000,23.200000,14.000000,12.300000,20.600000,24.400000,22.100000,97.500000,5.000000,76.892501,24.400000,16.100000,49.700000,20.600000,16.600000,23.100000,30.900000,13.400000,6.688119,60.624497,40.663492,53.756426,45.788781,12.737706,0.0,3041.000000,45.0


In [ ]:
US_results[US_results['ball']==33].describe()

,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016,vote_gains,seat_gains,point,ball
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.0
mean,24.354545,32.600000,21.554545,7.045455,14.459091,27.631818,8.322727,12.722727,17.809091,19.159091,14.345455,85.390909,1.627273,100.0,38.540909,13.440909,47.450000,20.850000,12.877273,18.831818,31.195455,17.250000,3.854738,24.068023,74.286328,19.277524,75.830504,1.544176,4.545455,1792.772727,33.0
std,6.452148,5.917287,4.551999,2.567142,4.086407,3.792899,1.715734,2.679185,1.371447,2.170288,2.767279,8.095731,0.989075,0.0,5.946160,4.387717,5.031591,2.924405,2.764560,3.957920,2.680703,2.879443,1.223743,11.368069,11.759690,7.614406,8.706103,4.198965,21.320072,949.519625,0.0
min,12.100000,19.000000,10.100000,2.800000,7.700000,19.000000,2.600000,9.600000,14.800000,15.600000,10.500000,72.900000,0.500000,100.0,25.700000,3.300000,38.200000,14.400000,7.000000,11.600000,26.400000,12.100000,2.311796,3.448276,48.711755,3.144654,57.859327,-6.213469,0.000000,75.000000,33.0
25%,19.825000,27.525000,19.200000,5.150000,11.275000,25.575000,7.725000,10.975000,17.025000,17.375000,12.525000,79.475000,0.925000,100.0,36.625000,11.900000,44.700000,19.100000,11.825000,16.550000,29.500000,15.150000,3.119069,17.057145,67.926855,14.020677,72.237344,-1.056930,0.000000,898.750000,33.0
50%,23.900000,33.850000,21.550000,7.050000,13.800000,28.700000,8.650000,12.400000,18.000000,19.200000,13.800000,84.850000,1.200000,100.0,38.600000,13.150000,47.700000,20.650000,13.250000,18.400000,30.450000,17.450000,3.641321,22.020814,76.103307,19.070465,76.363766,0.844976,0.000000,2129.000000,33.0
75%,26.975000,36.800000,24.750000,9.200000,17.475000,29.500000,9.375000,13.900000,18.800000,20.525000,15.925000,92.000000,2.200000,100.0,40.650000,15.550000,51.175000,22.750000,14.300000,21.050000,33.475000,19.125000,4.249519,29.725602,81.449390,22.014309,80.469422,3.223609,0.000000,2666.500000,33.0
max,41.600000,42.500000,29.600000,10.900000,23.200000,33.600000,10.300000,22.300000,20.600000,23.300000,22.300000,99.100000,4.100000,100.0,52.300000,22.700000,58.300000,26.200000,18.400000,27.800000,37.500000,22.300000,7.072161,49.355878,95.862069,33.394495,93.710692,12.570194,100.000000,3028.000000,33.0


In [ ]:
states = ['IOWA', 'MICHIGAN', 'OHIO', 'PENNSYLVANIA', 'WISCONSIN','INDIANA','MISSOURI','WEST VIRGINIA']
US_results[(US_results['state'].isin(states))&(US_results['ball']==40)].describe()

,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016,vote_gains,seat_gains,point,ball
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.0
mean,11.572549,43.650980,20.039216,9.605882,15.150980,22.217647,7.178431,10.082353,16.941176,22.952941,20.619608,96.780392,1.270588,99.684363,30.805882,13.554902,55.088235,20.062745,11.694118,13.139216,32.047059,14.541176,5.639427,39.413546,58.528056,25.459171,69.932165,11.404110,15.686275,1564.882353,40.0
std,2.975707,3.466086,2.493999,2.691053,2.365364,2.507804,0.651249,0.798174,1.097666,1.711649,2.168319,2.462277,0.844108,2.050784,4.685911,3.331085,4.937637,1.958873,2.115033,2.739568,1.686221,2.630223,1.439194,8.906570,8.680180,6.560954,7.407333,3.196059,36.729002,857.859467,0.0
min,6.700000,32.900000,13.300000,4.100000,9.700000,12.400000,5.800000,8.000000,14.600000,19.900000,16.000000,85.700000,0.400000,85.375531,22.100000,9.000000,46.100000,15.000000,4.300000,6.800000,27.600000,10.600000,3.428662,21.329305,38.643703,12.378223,53.850091,5.256924,0.000000,673.000000,40.0
25%,9.650000,41.800000,18.550000,7.750000,13.400000,21.150000,6.800000,9.650000,16.200000,21.950000,19.150000,96.350000,0.800000,100.000000,27.550000,11.000000,51.750000,19.050000,10.400000,11.350000,31.050000,12.450000,4.642057,33.417355,53.273157,20.841515,64.487916,9.424023,0.000000,821.000000,40.0
50%,11.400000,43.400000,20.500000,9.300000,15.100000,22.700000,7.100000,10.200000,16.800000,22.500000,20.300000,97.600000,0.900000,100.000000,30.000000,13.200000,53.700000,20.000000,11.800000,13.500000,32.200000,13.900000,5.461302,39.261800,58.654689,24.308481,70.374150,11.050191,0.000000,1461.000000,40.0
75%,12.700000,45.900000,22.000000,11.450000,16.650000,23.450000,7.600000,10.550000,17.600000,23.800000,22.350000,98.200000,1.350000,100.000000,34.150000,14.950000,58.150000,21.200000,13.150000,15.100000,32.950000,16.100000,6.688871,44.758096,63.867926,29.891032,75.762124,13.236241,0.000000,2153.000000,40.0
max,19.700000,49.900000,24.600000,15.000000,21.600000,29.400000,9.600000,11.700000,19.800000,28.400000,26.900000,98.900000,3.900000,100.000000,46.100000,22.900000,68.400000,24.900000,16.500000,19.300000,36.100000,22.300000,9.371362,59.590958,76.888270,41.213894,85.157593,22.810710,100.000000,3088.000000,40.0


In [ ]:
states = ['IOWA', 'MICHIGAN', 'OHIO', 'PENNSYLVANIA', 'WISCONSIN','INDIANA','MISSOURI','WEST VIRGINIA']
states = ['MICHIGAN', 'PENNSYLVANIA', 'WISCONSIN']

US_results[US_results['state'].isin(states)]

,GEO_ID,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Black,Rural,manu_arg,Poverty,lessthan50K,50to75k,75to100k,morethan100k,FIPS,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value,county_name,state,DEMOCRAT_2012,REPUBLICAN_2012,DEMOCRAT_2016,REPUBLICAN_2016,vote_gains,seat_gains,point,ball
4070,26001,11.5,40.2,25.3,8.6,14.5,13.2,5.1,6.2,13.4,27.8,34.5,98.4,0.8,89.808048,19.0,14.9,64.3,18.9,9.3,7.5,26001,30.7,13.0,10.453598,ALCONA,MICHIGAN,40.498034,58.502621,27.944498,67.779929,9.277308,0,1204,18
4071,26001,11.5,40.2,25.3,8.6,14.5,13.2,5.1,6.2,13.4,27.8,34.5,98.4,0.8,89.808048,19.0,14.9,64.3,18.9,9.3,7.5,26001,30.7,13.0,10.453598,ALCONA,MICHIGAN,40.498034,58.502621,27.944498,67.779929,9.277308,0,1204,42
4072,26003,10.6,44.3,19.1,7.9,18.1,15.5,7.4,10.4,17.5,25.8,23.4,88.5,7.1,100.000000,16.7,13.1,58.6,18.4,12.8,10.2,26003,32.3,13.3,10.256410,ALGER,MICHIGAN,47.899524,50.454742,36.808322,57.215582,6.760840,0,1205,2
4073,26003,10.6,44.3,19.1,7.9,18.1,15.5,7.4,10.4,17.5,25.8,23.4,88.5,7.1,100.000000,16.7,13.1,58.6,18.4,12.8,10.2,26003,32.3,13.3,10.256410,ALGER,MICHIGAN,47.899524,50.454742,36.808322,57.215582,6.760840,0,1205,18
4074,26003,10.6,44.3,19.1,7.9,18.1,15.5,7.4,10.4,17.5,25.8,23.4,88.5,7.1,100.000000,16.7,13.1,58.6,18.4,12.8,10.2,26003,32.3,13.3,10.256410,ALGER,MICHIGAN,47.899524,50.454742,36.808322,57.215582,6.760840,0,1205,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10671,55137,13.8,41.8,21.8,7.6,15.1,18.8,6.4,10.1,17.1,25.3,22.4,94.4,2.0,99.714480,27.8,11.5,53.5,20.7,11.7,14.1,55137,32.0,13.5,7.160494,WAUSHARA,WISCONSIN,44.281208,54.465471,31.602634,63.913659,9.448188,0,3088,42
10672,55139,7.9,34.0,20.8,10.4,26.9,20.9,12.3,13.1,18.6,20.2,14.7,91.1,4.1,17.613024,25.6,12.2,46.0,21.6,13.4,19.0,55139,28.1,12.3,5.305674,WINNEBAGO,WISCONSIN,50.967221,47.236271,42.520426,49.857707,2.621436,100,3089,0
10673,55139,7.9,34.0,20.8,10.4,26.9,20.9,12.3,13.1,18.6,20.2,14.7,91.1,4.1,17.613024,25.6,12.2,46.0,21.6,13.4,19.0,55139,28.1,12.3,5.305674,WINNEBAGO,WISCONSIN,50.967221,47.236271,42.520426,49.857707,2.621436,100,3089,25
10674,55141,7.6,38.9,21.0,12.1,20.4,21.9,7.4,11.4,17.7,23.0,18.6,95.1,1.6,33.115286,22.9,10.6,50.1,20.5,13.0,16.5,55141,34.2,12.3,6.072165,WOOD,WISCONSIN,47.766067,50.652956,37.650794,56.886243,6.233287,0,3090,0


In [ ]:
import pandas as pd

# Example DataFrame (replace this with actual data)
# US_results = pd.read_csv('path_to_your_data.csv')

# List of states to analyze
states = ['IOWA', 'MICHIGAN', 'OHIO', 'PENNSYLVANIA', 'WISCONSIN', 'INDIANA', 'MISSOURI', 'WEST VIRGINIA']
states = ['MICHIGAN', 'PENNSYLVANIA', 'WISCONSIN']

# Calculate the total occurrences of each unique value in the 'ball' column
total_ball_counts = US_results['ball'].value_counts()

# Filter the DataFrame to include only rows from the specified states
filtered_data = US_results[US_results['state'].isin(states)]

# Calculate the occurrences of each unique value in the 'ball' column for the filtered data
filtered_ball_counts = filtered_data['ball'].value_counts()

# Compute the proportion of each ball that belongs to the specified states
ball_proportions = filtered_ball_counts / total_ball_counts

# Convert the proportions to a DataFrame for better display
ball_proportions_df = ball_proportions.to_frame(name='Proportion')

# Fill NaN values with 0 to indicate that the ball did not appear in the filtered data
ball_proportions_df.fillna(0, inplace=True)

# Display the result
print(ball_proportions_df.sort_values(by='Proportion'))


      Proportion
ball            
58      0.000000
39      0.000000
38      0.000000
36      0.000000
48      0.000000
49      0.000000
50      0.000000
31      0.000000
57      0.000000
51      0.000000
52      0.000000
53      0.000000
47      0.000000
55      0.000000
16      0.000000
20      0.000000
14      0.000000
1       0.000000
9       0.000000
3       0.000000
4       0.000000
8       0.000000
56      0.000000
7       0.000000
12      0.005435
24      0.011111
54      0.012579
11      0.018182
13      0.021583
21      0.023474
37      0.026316
19      0.032000
28      0.033333
6       0.043825
33      0.045455
15      0.045802
27      0.047170
30      0.055556
41      0.056034
23      0.056604
2       0.060738
10      0.061947
29      0.068966
44      0.070588
18      0.084046
25      0.086957
0       0.088553
34      0.090361
5       0.093230
17      0.093333
42      0.100559
26      0.100629
46      0.104167
22      0.107042
32      0.107452
43      0.111111
40      0.1456

In [ ]:
bmx1.columns

Index(['Nohighschool_25plus', 'Highschool_25plus', 'Somedegree_25plus',
       'Assocdegree_25plus', 'Degree_25plus', 'Under18', '18to24', '25to34',
       '35to49', '50to64', 'Over65', 'White', 'Rural', 'lessthan50K',
       '50to75k', '75to100k', 'morethan100k', 'manu_arg', 'Poverty',
       'Adult obesity raw value', 'Poor or fair health raw value',
       'Unemployment raw value'],
      dtype='object')

In [ ]:
columns=['REPUBLICAN_2012','REPUBLICAN_2016','Nohighschool_25plus', 'Highschool_25plus', 'Somedegree_25plus',
       'Assocdegree_25plus', 'Degree_25plus', 'Under18', '18to24', '25to34',
       '35to49', '50to64', 'Over65', 'White', 'Rural', 'lessthan50K',
       '50to75k', '75to100k', 'morethan100k', 'manu_arg', 'Poverty',
       'Adult obesity raw value', 'Poor or fair health raw value',
       'Unemployment raw value']
ball_means=pd.DataFrame()
for col in columns:
    ball_means[col]=US_results.groupby('ball')[col].mean()

In [ ]:
ball_means

,REPUBLICAN_2012,REPUBLICAN_2016,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Rural,lessthan50K,50to75k,75to100k,morethan100k,manu_arg,Poverty,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value
ball,,,,,,,,,,,,,,,,,,,,,,,,
0,60.149892,63.292547,12.457019,33.490497,22.833909,9.179806,22.040713,23.035637,9.614039,12.144816,18.221706,20.545356,16.431102,85.680022,39.950430,50.145572,19.293629,12.557667,18.007559,17.634665,14.933153,30.781102,15.901944,5.824436
1,53.497064,56.907903,20.928205,36.996795,20.345513,7.348077,14.382692,23.521154,9.396795,12.088462,18.266667,20.335256,16.380128,58.548077,63.182478,63.775000,16.132692,9.079487,11.004487,20.664744,24.609615,35.869872,23.375641,8.258201
2,65.727843,72.156954,16.990130,39.057592,21.283623,7.740672,14.926139,22.103905,7.870390,10.877115,17.897072,21.851193,19.399024,87.117896,96.734544,58.608785,18.338720,10.573319,12.481887,22.314751,18.008026,32.456941,18.239479,6.501271
3,33.833089,37.005677,25.185106,37.189362,18.608511,6.629787,12.404255,22.853191,9.585106,12.274468,17.855319,20.827660,16.604255,35.091489,98.888304,71.925532,13.823404,7.034043,7.219149,24.940426,31.702128,38.761702,28.131915,10.732063
4,51.550907,55.738564,21.963793,39.114943,19.594828,6.804598,12.520690,22.389080,8.894253,11.779310,18.163218,21.190805,17.587356,57.185632,96.598167,66.250575,15.683333,8.435057,9.625862,22.532184,25.738506,35.658621,23.923563,8.568748
5,63.291160,69.101354,15.449501,37.696115,21.504329,8.469367,16.884129,22.498557,8.709767,11.338513,18.197891,21.280355,17.973807,87.926748,67.253498,55.955605,18.865705,11.329301,13.851165,20.680466,17.211210,32.164262,17.587791,6.626545
6,47.669247,46.448863,13.453386,27.874104,22.503586,8.262550,27.908765,23.770518,10.827888,14.271315,18.819522,18.818327,13.495219,64.193227,11.480452,49.073705,18.094821,12.003187,20.830279,11.311155,17.123506,29.613147,17.686056,6.497264
7,33.430072,36.035029,22.058333,33.370833,21.308333,7.908333,15.366667,24.383333,10.400000,12.483333,17.650000,20.037500,15.054167,34.916667,52.817042,68.454167,14.700000,7.783333,9.037500,18.600000,30.695833,38.737500,26.575000,10.396488
8,53.933766,61.656207,15.600000,42.000000,23.300000,7.000000,11.900000,12.300000,10.500000,13.600000,29.100000,27.800000,6.700000,25.100000,100.000000,35.700000,21.800000,15.600000,26.900000,69.600000,15.500000,31.100000,17.000000,4.585931


In [ ]:
ball_means.sort_values(by='White',ascending=True)

,REPUBLICAN_2012,REPUBLICAN_2016,Nohighschool_25plus,Highschool_25plus,Somedegree_25plus,Assocdegree_25plus,Degree_25plus,Under18,18to24,25to34,35to49,50to64,Over65,White,Rural,lessthan50K,50to75k,75to100k,morethan100k,manu_arg,Poverty,Adult obesity raw value,Poor or fair health raw value,Unemployment raw value
ball,,,,,,,,,,,,,,,,,,,,,,,,
8,53.933766,61.656207,15.600000,42.000000,23.300000,7.000000,11.900000,12.300000,10.500000,13.600000,29.100000,27.800000,6.700000,25.100000,100.000000,35.700000,21.800000,15.600000,26.900000,69.600000,15.500000,31.100000,17.000000,4.585931
11,32.173088,35.584557,21.481818,35.807273,20.661818,7.889091,14.181818,26.481818,10.423636,12.001818,16.934545,19.547273,14.623636,31.347273,98.705752,69.269091,14.538182,7.585455,8.610909,18.850909,33.478182,38.947273,27.660000,10.438253
30,27.240753,26.699556,17.022222,29.777778,22.355556,7.044444,23.816667,24.122222,10.455556,14.955556,18.350000,19.061111,13.044444,33.527778,5.909676,58.472222,16.661111,10.066667,14.777778,9.716667,24.183333,33.988889,22.777778,8.287212
7,33.430072,36.035029,22.058333,33.370833,21.308333,7.908333,15.366667,24.383333,10.400000,12.483333,17.650000,20.037500,15.054167,34.916667,52.817042,68.454167,14.700000,7.783333,9.037500,18.600000,30.695833,38.737500,26.575000,10.396488
3,33.833089,37.005677,25.185106,37.189362,18.608511,6.629787,12.404255,22.853191,9.585106,12.274468,17.855319,20.827660,16.604255,35.091489,98.888304,71.925532,13.823404,7.034043,7.219149,24.940426,31.702128,38.761702,28.131915,10.732063
9,40.050132,43.074233,12.500000,40.300000,25.100000,6.400000,15.700000,15.700000,11.600000,18.500000,25.300000,23.200000,5.700000,37.800000,100.000000,26.400000,15.900000,19.200000,38.600000,39.100000,7.700000,36.200000,14.000000,4.933243
39,30.183303,27.591384,14.080000,25.300000,19.490000,7.390000,33.730000,24.310000,9.330000,15.190000,21.830000,18.370000,10.980000,38.390000,2.236622,36.460000,18.880000,13.500000,31.190000,5.830000,12.450000,26.670000,15.830000,6.026485
14,47.847441,49.403716,18.731132,34.104717,21.881132,7.676415,17.613208,24.565094,10.083019,13.027358,18.120755,19.535849,14.683019,51.151887,36.687256,61.302830,16.400943,9.610377,12.683962,16.964151,24.744340,35.512264,23.052830,8.558139
38,43.416019,42.062505,9.688889,28.322222,23.177778,8.822222,30.011111,25.488889,8.655556,12.811111,21.311111,19.755556,11.933333,55.655556,23.947975,31.177778,17.711111,14.988889,36.133333,6.877778,9.088889,29.011111,13.688889,5.383683


In [ ]:
ball_means.to_csv('Outputs/US_ballmeans.csv')